In [ ]:
import glob
import json
import pandas as pd
from datetime import datetime
import re

def process_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    # Список для зберігання оброблених даних
    processed_data = []
    
    for message in data.get('messages', []):
        if message['type'] == 'message':
            try:
                # Витягуємо дату та час
                message_date = datetime.fromisoformat(message['date'])
                
                # Витягуємо текст повідомлення
                message_text = ' '.join([item['text'] if isinstance(item, dict) and 'text' in item else item for item in message['text']])

                # Шукаємо хештеги, щоб визначити місце
                hashtag_matches = re.findall(r"#([А-Яа-яІіЇїЄєҐґ0-9_]*)", message_text)
                if hashtag_matches:
                    # Для кожного знайденого хештегу замінюємо нижнє підкреслення на пробіл і видаляємо сам символ "#"
                    place = ' '.join([match.replace("_", " ") for match in hashtag_matches])
                else:
                    place = None
                
                # Додаємо точку після "м", якщо її немає (скорочення для "місто")
                if place and place.startswith("м ") and not place.startswith("м. "):
                    place = "м. " + place[2:]

                # Перевіряємо тип повідомлення (Повітряна тривога або Відбій тривоги)
                message_type = "Повітряна тривога" if "🔴" in message_text else "Відбій тривоги"

                # Додаємо рядок в результат
                processed_data.append({
                    'Дата та час': message_date,
                    'Місце': place,
                    'Тип повідомлення': message_type
                })
            except Exception as e:
                print(f"Помилка обробки повідомлення: {e}")
                
    return processed_data

def process_multiple_json_files(directory_path):
    # Шукаємо всі JSON файли в директорії
    file_paths = glob.glob(f"{directory_path}/*.json")
    
    all_data = []
    for file_path in file_paths:
        all_data.extend(process_json_file(file_path))
    
    # Створення DataFrame з усієї інформації
    df = pd.DataFrame(all_data)
    
    return df

# Вказуємо шлях до директорії з JSON файлами
directory_path = './archive'

# Обробка файлів
final_df = process_multiple_json_files(directory_path)

# Збереження результату в CSV
final_df.to_csv('processed_data.csv', index=False)

# Вивести перші кілька рядків для перевірки
print(final_df.head())


          Дата та час                                              Місце  \
0 2022-04-10 00:04:39                                 Харківська область   
1 2022-04-10 00:46:59                                 Харківська область   
2 2022-04-10 01:05:47                           Дніпропетровська область   
3 2022-04-10 01:06:24  м. Кривий Ріг та Криворізька територіальна гро...   
4 2022-04-10 01:29:23                           Дніпропетровська область   

    Тип повідомлення  
0     Відбій тривоги  
1  Повітряна тривога  
2  Повітряна тривога  
3  Повітряна тривога  
4     Відбій тривоги  
